In [27]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations
from collections import defaultdict
import numpy as np
from memory_profiler import memory_usage

In [28]:
#Load the dataset
file_path = '/Users/srujana_chintala/Desktop/Data Mining/Project/processed_survey_data_columns.csv'
df = pd.read_csv(file_path)

In [29]:
df_encoded = pd.get_dummies(df)
print(df_encoded)
df_encoded = df_encoded.astype(int)

      ID  Gender  Age_Group  Residence  Education_Level  Source_of_Income  \
0      1       1          1          2                4                 1   
1      2       1          2          4                3                 1   
2      3       1          2          2                4                 1   
3      4       2          4          4                3                 3   
4      5       2          3          1                2                 1   
..   ...     ...        ...        ...              ...               ...   
295  296       1          2          4                3                 1   
296  297       2          3          3                2                 1   
297  298       1          3          2                2                 1   
298  299       2          3          4                3                 1   
299  300       1          2          3                4                 4   

     Marital_Status  Smoked_Cigarettes  Year_Diagnosed  Surgical_Treatment 

In [30]:
'''
unique_values = df_encoded.nunique()
print(unique_values)

df_encoded.value_counts()
'''

'\nunique_values = df_encoded.nunique()\nprint(unique_values)\n\ndf_encoded.value_counts()\n'

In [31]:
df_select_1hot = pd.get_dummies(df_encoded)
df_select_1hot = df_select_1hot.astype(int)
print(df_select_1hot)

df_select_1hot.head(20)

      ID  Gender  Age_Group  Residence  Education_Level  Source_of_Income  \
0      1       1          1          2                4                 1   
1      2       1          2          4                3                 1   
2      3       1          2          2                4                 1   
3      4       2          4          4                3                 3   
4      5       2          3          1                2                 1   
..   ...     ...        ...        ...              ...               ...   
295  296       1          2          4                3                 1   
296  297       2          3          3                2                 1   
297  298       1          3          2                2                 1   
298  299       2          3          4                3                 1   
299  300       1          2          3                4                 4   

     Marital_Status  Smoked_Cigarettes  Year_Diagnosed  Surgical_Treatment 

,ID,Gender,Age_Group,Residence,Education_Level,Source_of_Income,Marital_Status,Smoked_Cigarettes,Year_Diagnosed,Surgical_Treatment,...,Daily_Life_Impact_family_life,Daily_Life_Impact_social_life,Daily_Life_Impact_no_effect,Energy_Level,Self_Care,Daily_Activities_Difficulty,Work_Readiness,Support_Satisfaction,Coping_Strategy,Negative_Emotions
0,1,1,1,2,4,1,1,1,2021,0,...,0,0,0,2,1,1,2,2,3,3
1,2,1,2,4,3,1,2,1,2020,1,...,0,0,0,2,1,2,2,2,3,3
2,3,1,2,2,4,1,2,1,2020,1,...,0,0,0,2,1,1,2,1,3,2
3,4,2,4,4,3,3,2,1,2021,1,...,0,0,0,4,2,3,3,1,3,4
4,5,2,3,1,2,1,2,1,2019,1,...,0,0,0,2,1,1,3,3,3,3
5,6,2,3,3,3,1,2,1,2021,0,...,0,1,0,2,3,1,3,1,3,3
6,7,1,4,4,4,3,4,0,2022,0,...,1,0,0,2,1,1,3,1,3,3
7,8,2,1,3,4,1,2,1,2021,0,...,0,0,0,2,1,1,2,1,3,3
8,9,2,3,2,2,1,4,0,2020,0,...,0,0,0,3,2,2,2,2,3,3
9,10,2,2,2,2,1,3,1,2022,1,...,0,0,0,2,1,1,2,2,2,2


In [32]:
import pandas as pd
from itertools import combinations

# Convert the data into transaction lists (TID lists)
def get_tid_lists(df_select_1hot):
    tid_lists = {}
    for col in df_select_1hot.columns:
        tid_lists[col] = list(df_select_1hot[df_select_1hot[col] == 1].index)  # Get TIDs where item appears
    return tid_lists

# Function to calculate support
def support(tid_list, total_transactions):
    return len(tid_list) / total_transactions

# Function to calculate Lift
def lift(tid_list, total_transactions, itemset, tid_lists):
    # Support of itemset
    itemset_support = support(tid_list, total_transactions)
    
    # Support of individual items in the itemset
    individual_supports = [support(tid_lists[item], total_transactions) for item in itemset]
    
    # Expected support if items were independent
    expected_support = 1
    for item_support in individual_supports:
        expected_support *= item_support
    
    # Lift: observed support / expected support
    if expected_support != 0:
        return itemset_support / expected_support
    return 0

# Function to calculate Conviction
def conviction(tid_list, total_transactions, itemset, tid_lists):
    itemset_support = support(tid_list, total_transactions)
    # Calculate support for each item in the itemset
    individual_supports = [support(tid_lists[item], total_transactions) for item in itemset]
    # Calculate Conviction
    expected_support = 1
    for item_support in individual_supports:
        expected_support *= item_support
    
    if expected_support != 1:
        return (1 - itemset_support) / (1 - expected_support)
    return float('inf')

# Function to calculate Leverage
def leverage(tid_list, total_transactions, itemset, tid_lists):
    itemset_support = support(tid_list, total_transactions)
    # Calculate support for each item in the itemset
    individual_supports = [support(tid_lists[item], total_transactions) for item in itemset]
    # Expected support if items were independent
    expected_support = 1
    for item_support in individual_supports:
        expected_support *= item_support
    # Leverage: observed support - expected support
    return itemset_support - expected_support

# Implement the ECLAT algorithm with pattern evaluation metrics
def eclat(df_select_1hot, min_support=0.5):
    tid_lists = get_tid_lists(df_select_1hot)
    total_transactions = len(df_select_1hot)
    
    frequent_itemsets = []

    # Generate all single-itemsets
    for item, tid_list in tid_lists.items():
        if support(tid_list, total_transactions) >= min_support:
            frequent_itemsets.append((frozenset([item]), tid_list))
    
    # Generate larger itemsets (combinations of two or more items)
    itemsets_to_check = list(combinations(tid_lists.keys(), 2))
    for itemset in itemsets_to_check:
        tid_list_intersection = list(set(tid_lists[itemset[0]]) & set(tid_lists[itemset[1]]))
        if support(tid_list_intersection, total_transactions) >= min_support:
            frequent_itemsets.append((frozenset(itemset), tid_list_intersection))
    
    # For larger itemsets (3-itemsets, 4-itemsets, etc.)
    k = 3
    while True:
        new_itemsets = []
        for itemset in combinations(tid_lists.keys(), k):
            # Calculate intersection of TIDs of k items
            tid_list_intersection = set(tid_lists[itemset[0]])
            for item in itemset[1:]:
                tid_list_intersection &= set(tid_lists[item])
            
            if support(list(tid_list_intersection), total_transactions) >= min_support:
                new_itemsets.append((frozenset(itemset), list(tid_list_intersection)))
        
        if not new_itemsets:
            break
        
        frequent_itemsets.extend(new_itemsets)
        k += 1
    
    return frequent_itemsets

# Apply the ECLAT algorithm with a minimum support of 0.5
min_support = 0.5
frequent_itemsets = eclat(df_select_1hot, min_support)

# Initialize an empty list to store the evaluation results
evaluation_results = []

# Iterate through each frequent itemset and calculate the evaluation metrics
for itemset, tid_list in frequent_itemsets:
    itemset_support = support(tid_list, len(df_select_1hot))
    itemset_lift = lift(tid_list, len(df_select_1hot), itemset, get_tid_lists(df_select_1hot))
    itemset_conviction = conviction(tid_list, len(df_select_1hot), itemset, get_tid_lists(df_select_1hot))
    itemset_leverage = leverage(tid_list, len(df_select_1hot), itemset, get_tid_lists(df_select_1hot))
    
    # Append the results as a dictionary
    evaluation_results.append({
        'Itemset': itemset,
        'Support': round(itemset_support, 4),
        'Lift': round(itemset_lift, 4),
        'Conviction': round(itemset_conviction, 4),
        'Leverage': round(itemset_leverage, 4)
    })

# Convert the evaluation results into a DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# Display the results as a table
print(evaluation_df)


                                              Itemset  Support    Lift  \
0                                            (Gender)   0.5400  1.0000   
1                                  (Source_of_Income)   0.5800  1.0000   
2                                 (Smoked_Cigarettes)   0.6267  1.0000   
3                                      (Chemotherapy)   0.5867  1.0000   
4                                  (Emotional_Impact)   0.8633  1.0000   
5                                     (Travel_Impact)   0.6800  1.0000   
6                                (Support_From_Close)   0.9033  1.0000   
7                               (Health_Satisfaction)   0.7000  1.0000   
8                   (Daily_Life_Impact_Psychological)   0.5467  1.0000   
9                                         (Self_Care)   0.6167  1.0000   
10                             (Support_Satisfaction)   0.7767  1.0000   
11                       (Support_From_Close, Gender)   0.5067  1.0387   
12             (Support_From_Close, So

In [33]:
import pandas as pd
import itertools


def get_tid_lists(df_select_1hot):
    """Assuming df is a one-hot encoded DataFrame, return the TID lists for each item."""
    tid_lists = {}
    for column in df_select_1hot.columns:
        tid_lists[column] = set(df_select_1hot.index[df_select_1hot[column] == 1].tolist())
    return tid_lists

def support(tid_list, total_transactions):
    """Calculate support as the fraction of transactions containing the itemset."""
    return len(tid_list) / total_transactions

def generate_association_rules(frequent_itemsets, df_select_1hot, min_confidence=0.7):
    association_rules = []
    
    # Get the TID lists for all items in the dataset
    tid_lists = get_tid_lists(df_select_1hot)
    total_transactions = len(df_select_1hot)

    # Iterate through each frequent itemset
    for itemset, tid_list in frequent_itemsets.items():
        if len(itemset) > 1:
            # Generate all possible splits of the itemset into LHS (left-hand side) and RHS (right-hand side)
            for i in range(1, len(itemset)):
                lhs_combinations = itertools.combinations(itemset, i)
                for lhs in lhs_combinations:
                    lhs = frozenset(lhs)  # Convert to frozenset to ensure it's hashable
                    
                    rhs = itemset - lhs  # The rest of the itemset is the RHS (right-hand side)

                    if rhs:
                        # Get the TID list for LHS
                        lhs_tid_list = set()
                        for item in lhs:
                            lhs_tid_list.update(tid_lists.get(item, set()))

                        # Get the TID list for RHS
                        rhs_tid_list = set()
                        for item in rhs:
                            rhs_tid_list.update(tid_lists.get(item, set()))

                        # Calculate confidence
                        if lhs_tid_list:
                            # TIDs where both LHS and RHS are present
                            both_tid_list = lhs_tid_list & rhs_tid_list
                            confidence = len(both_tid_list) / len(lhs_tid_list)

                            if confidence >= min_confidence:
                                # Add the rule to the association_rules list
                                association_rules.append({
                                    'Rule': f'{lhs} -> {rhs}',
                                    'Support': support(tid_list, total_transactions),
                                    'Confidence': confidence
                                })
    
    # Return the association rules as a DataFrame
    return pd.DataFrame(association_rules)


In [34]:
# Convert the list to a dictionary (key: itemset, value: TID list)
frequent_itemsets_dict = {itemset: tid_list for itemset, tid_list in frequent_itemsets}

# Now pass the dictionary to the generate_association_rules function
rules_df = generate_association_rules(frequent_itemsets_dict, df_select_1hot, min_confidence=0.7)

# Display the generated association rules
if not rules_df.empty:
    print("Association Rules:")
    print(rules_df)
else:
    print("No association rules found.")

Association Rules:
                                                  Rule   Support  Confidence
0    frozenset({'Gender'}) -> frozenset({'Support_F...  0.506667    0.938272
1    frozenset({'Source_of_Income'}) -> frozenset({...  0.516667    0.890805
2    frozenset({'Smoked_Cigarettes'}) -> frozenset(...  0.573333    0.914894
3    frozenset({'Smoked_Cigarettes'}) -> frozenset(...  0.586667    0.936170
4    frozenset({'Chemotherapy'}) -> frozenset({'Emo...  0.563333    0.960227
..                                                 ...       ...         ...
136  frozenset({'Emotional_Impact', 'Support_Satisf...  0.550000    0.992958
137  frozenset({'Health_Satisfaction', 'Support_Fro...  0.550000    0.809028
138  frozenset({'Health_Satisfaction', 'Support_Fro...  0.550000    0.898601
139  frozenset({'Health_Satisfaction', 'Emotional_I...  0.550000    0.940351
140  frozenset({'Support_From_Close', 'Emotional_Im...  0.550000    0.728223

[141 rows x 3 columns]


In [35]:
import time

# Start the high-resolution timer
start_time = time.perf_counter()

# Run the Eclat algorithm
frequent_itemsets = eclat(df_select_1hot)

# Stop the timer
end_time = time.perf_counter()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time:.6f} seconds")

Execution time: 5.010191 seconds


In [36]:
# Define a wrapper function to call the ECLAT algorithm
def run_eclat():
    min_support = 0.6
    frequent_itemsets = eclat(df_select_1hot, min_support)
    return frequent_itemsets

# Check memory usage
mem_usage = memory_usage(run_eclat, interval=0.1, timeout=None)
print(f"Memory usage: {max(mem_usage) - min(mem_usage)} MiB")


Memory usage: 1.046875 MiB
